In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os

In [31]:
os.chdir(os.getcwd())
print(os.getcwd())

c:\Users\Krzysztof\Desktop\ensembleAI-ScoutTeam


In [32]:
path = os.getcwd()
print(path)
os.chdir('..')
print(os.getcwd())

dataset_t1 = torch.load("task_1_modelstealing/data/ModelStealingPub.pt")

/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack
/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam


In [6]:
os.chdir(path)
print(os.getcwd())
dataset = torch.load("data/SybilAttack.pt")

/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack


In [7]:
set_t2 = set(dataset.ids)
set_t1 = set(dataset_t1.ids)

intersection = set_t2.intersection(set_t1)

print(intersection)


set()


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class Linear(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
l1_penalty = l1_weight * sum([p.abs().sum() for p in net.hidden.parameters()])